In [ ]:
#
# NCFApp
# New Customers Forecast Application
# Programma creato per il forecast dei clienti
# 

import numpy             as np 
import pandas            as pd 
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.figure import Figure

from Tkinter import *
import tkMessageBox

from PIL import ImageTk, Image
import os

from statsmodels.tsa.stattools    import acf, pacf, adfuller, arma_order_select_ic
from statsmodels.tsa.seasonal     import seasonal_decompose
from statsmodels.tsa.arima_model  import ARIMA, ARIMAResults
from statsmodels.stats.diagnostic import acorr_ljungbox

def reducto(timeserie) :
# Funzione che taglia la coda della serie storica e tiene i valori piu recenti
# Le serie vengono tagliate ai 4 anni più recenti e ...
# + 12 mesi per la differenziazione stagionale e ...
# + 1 mese per la differenziazione de-trend
    if len(timeserie)>61 :
        tms_object = timeserie[-61:]
    elif len(timeserie)>49 :
        tms_object = timeserie[-49:]
    elif len(timeserie)>37 :
        tms_object = timeserie[-37:]
    return tms_object

#def Componi(first_list,second_list) :
#    third_list = []
#    for first in first_list :
#        for second in second_list :
#            third_list.append(first+'_'+second)
#    return third_list

# Liste di uso comune e dizionario con le declinazioni dei codici

global Tipomisura
Tipomisura       = ('FL','CP','FN','ST')
global Tipomisura_file
Tipomisura_file  = ('FL','CP','ST')
global Tipomisura_fore
Tipomisura_fore  = ('FL','CP')
global Tipocliente
Tipocliente      = ('IMP','PER','AZR','RET','TOTRET','BDT')
global Tipocliente_file
Tipocliente_file = ('IMP','PER','AZR','RET')
global Tipocliente_fore
Tipocliente_fore = ('IMP','PER','AZR','RET')
global Tiposerie
Tiposerie        = ('CONS','FORE','TUTTO')
# Declinazioni
global Decl
Decl = {}
Decl['FL'] = 'FLUSSO LORDO'
Decl['CP'] = 'CLIENTI PERSI'
Decl['FN'] = 'FLUSSO NETTO'
Decl['ST'] = 'STOCK'
Decl['IMP'] = 'IMPRESE'
Decl['PER'] = 'PERSONAL'
Decl['AZR'] = 'AZIENDE RETAIL'
Decl['RET'] = 'PRIVATI RETAIL'
Decl['TOTRET'] = 'TOTALE RETAIL'
Decl['BDT'] = 'TOTALE BDT'
Decl['CONS'] = 'CONSUNTIVO'
Decl['FORE'] = 'FORECAST'
Decl['TUTTO'] = 'CONS.+FORECAST'

global alpha
alpha = 0.05
global mesi_forecast
mesi_forecast = 24



# Carica il dataframe dei dati dal file csv e converte la colonna 'MESE' da stringa a data
# Il dataframe contiene FL,CP e STOCK dei 4 tipi clienti : imprese,personal,aziende retail,privati retail
# i flussi netti sono calcolati per differenza e il totale BDT e retail per somma
global df_sstor
global df_cons
global df_pred
global df_fore
df_cons = pd.DataFrame()


dfp = pd.read_csv('NCFAdati.csv',sep=',',index_col=0, parse_dates=True, infer_datetime_format=True)

for txt1 in Tipomisura_file  :
    for txt2 in Tipocliente_file :
        df_cons[txt1+'_'+txt2] = reducto(dfp[txt1+'_'+txt2])

for txt in Tipocliente_file :
    df_cons['FN_'+txt] = df_cons['FL_'+txt] - df_cons['CP_'+txt]
    
for txt in Tipomisura :
    df_cons[txt+'_TOTRET'] = df_cons[txt+'_AZR'] + df_cons[txt+'_RET']
    df_cons[txt+'_BDT']    = df_cons[txt+'_IMP'] + df_cons[txt+'_PER'] + df_cons[txt+'_AZR'] + df_cons[txt+'_RET']
    

#df_sstor = pd.DataFrame()
df_sstor = df_cons.copy()
#listone = Componi(Tipomisura_file,Tipocliente_file)
#for txt in listone : df_sstor[txt] = reducto(dfp[txt])

global ultimo_consuntivo
#ultimo_consuntivo = max(df_sstor.index)
ultimo_consuntivo = max(df_cons.index)
global index_forecast
index_forecast = pd.date_range(ultimo_consuntivo, periods=1+mesi_forecast, freq='M')
index_forecast = index_forecast[1:]

df_pred = pd.DataFrame(columns=df_cons.columns, index = df_cons.index)
df_fore = pd.DataFrame(columns=df_cons.columns, index = index_forecast)

            
# Funzione che scrive una stringa nel file di log
def Wlog(stringa=None) :
    file_log.writelines([stringa,'\n'])

    
    # apre il file di log
file_log = open('NCFAlog.txt', mode='w')
Wlog('Aperta sessione di programma')




class Application:

    def __init__(self,master):
        #df_sstor = self.CreaDataframe
        #print df_sstor
        #self.df_sstor, self.ultimo_consuntivo, self.index_forecast = self.CreaDataframe
        #self.Graphic_init()
        
        self.frame1 = Frame(master)
        self.frame1.pack(side='top')
        
        self.img_logo = ImageTk.PhotoImage(Image.open("logobanca.png"))
        self.panel_logo = Label(self.frame1,image = self.img_logo, width=400)
        self.panel_logo.pack(side = "left", fill = "both", expand = "yes")
        
        self.pulsante1 = Button(self.frame1, text='Elaborazione FORECAST') 
        self.pulsante1["background"] = "white"
        self.pulsante1.bind('<Button-1>',self.ButtonForecast)
        self.pulsante1.pack(side='bottom')                              


        self.pulsante3 = Button(self.frame1, text='Log') 
        self.pulsante3["background"] = "green"
        self.pulsante3.bind('<Button-1>',self.ButtonLog)
        self.pulsante3.pack(side = 'right')                  
        
        self.pulsante2 = Button(self.frame1, text='Documentazione') 
        self.pulsante2["background"] = "cyan"
        self.pulsante2.bind('<Button-1>',self.ButtonDocumentazione)
        self.pulsante2.pack(side='bottom')                   
        
        self.label1 = Label(self.frame1,text = ' Ultimo consuntivo caricato '+str(ultimo_consuntivo)[0:-12]+' ')
        self.label1.width = 400
        self.label1.pack(side='bottom')
               
        self.frame2 = Frame(master)
        self.frame2.pack(side='top')
        
        self.label2 = Label(self.frame2,font=('Verdana',20))
        self.label2.pack(side='bottom', fill='both', expand=True)           
        
        self.frame3a = Frame(master, width=900)
        self.frame3a.pack(side='top')
        
        self.imgFL = Figure(figsize=(5,3), dpi=100)
        self.canvasFL = FigureCanvasTkAgg(self.imgFL, self.frame3a)
        self.canvasFL.show()
        self.canvasFL.get_tk_widget().pack(side='left', fill='both', expand=True)
        
        self.imgCP = Figure(figsize=(5,3), dpi=100)
        self.canvasCP = FigureCanvasTkAgg(self.imgCP, self.frame3a)
        self.canvasCP.show()
        self.canvasCP.get_tk_widget().pack(side='left', fill='both', expand=True)
        

        self.imgFN = Figure(figsize=(5,3), dpi=100)
        self.canvasFN = FigureCanvasTkAgg(self.imgFN, self.frame3a)
        self.canvasFN.show()
        self.canvasFN.get_tk_widget().pack(side='left', fill='both', expand=True)

        self.frame3b = Frame(master)
        self.frame3b.pack(side='top')
        
                
        self.imgST = Figure(figsize=(4,2.5), dpi=100)
        self.canvasST = FigureCanvasTkAgg(self.imgST, self.frame3b)
        self.canvasST.show()
        self.canvasST.get_tk_widget().pack(side='right', fill='both', expand=True)
        
                
        self.frame4 = Frame(master, width=900)
        self.frame4.pack(side='top')
        # Crea il gruppo dei pulsanti radio
        self.radiobase1 = StringVar()
        self.radiobase1.set('BDT')
        self.radiobase1.trace('w',self.ChangeGraph)
        self.radiobase2 = StringVar()
        self.radiobase2.trace('w',self.ChangeGraph)
        self.radiobase2.set('CONS')
        for code in Tipocliente :
            Radiobutton(self.frame4, text=Decl[code], variable=self.radiobase1, value=code,indicatoron=0).pack(anchor=W,side='left')
        for code in Tiposerie :
            Radiobutton(self.frame4,background='blue',foreground='white',
                        width=12, text=Decl[code], variable=self.radiobase2, value=code, indicatoron=0).pack(anchor=W,side='right')
        
        self.frame5 = Frame(master)
        self.frame5.pack(side='top')
        self.label5a = Label(self.frame5,text = ' ')
        self.label5a.pack(fill = "both", expand = "yes")
        self.label5b = Label(self.frame5,text = 'Applicazione per il forecast del flusso clienti - Release 1.0 dicembre 2017 - (C) Gianni Zattoni')
        self.label5b.pack(fill = "both", expand = "yes")
        self.label5c = Label(self.frame5,text = ' ')
        self.label5c.pack(fill = "both", expand = "yes")
        
    def ButtonForecast(self,evento) :
#        self.pulsante1["text"] = "Waiting..."     
#        self.pulsante1["background"] = "black"
        self.ForecastManager()
#        self.pulsante1["text"] = "Elaborazione FORECAST"     
#        self.pulsante1["background"] = "white"
        pass

    def ButtonDocumentazione(self,evento) :
        pass
    
    def ButtonLog(self,evento):
        pass
    
    def RefreshGraph(self, tipocliente, tiposerie):
        self.label2['text'] = Decl[tiposerie]+' - '+Decl[tipocliente]
        
        header = 'FL_'+tipocliente
        self.imgFL.clear()
        ax = self.imgFL.add_subplot(111,title=Decl['FL'])
        if tiposerie == 'CONS' or tiposerie=='TUTTO' :
            ax.plot(df_cons[header])
            ax.plot(df_cons[header].rolling(window=12,center=False).mean())
            ax.plot(df_pred[header])
        if tiposerie == 'FORE' or tiposerie=='TUTTO' :
            ax.plot(df_fore[header])
            ax.plot(df_pred[header])
        self.canvasFL.draw()
        
        header = 'CP_'+tipocliente
        self.imgCP.clear()
        ax = self.imgCP.add_subplot(111,title=Decl['CP'])
        if tiposerie == 'CONS' or tiposerie=='TUTTO' :
            ax.plot(df_cons[header])
            ax.plot(df_cons[header].rolling(window=12,center=False).mean())
        if tiposerie == 'FORE' or tiposerie=='TUTTO' :
            ax.plot(df_fore[header])
        self.canvasCP.draw()
        
        header = 'FN_'+tipocliente
        self.imgFN.clear()
        ax = self.imgFN.add_subplot(111,title=Decl['FN'])
        if tiposerie == 'CONS' or tiposerie=='TUTTO' :
            ax.plot(df_cons[header])
            ax.plot(df_cons[header].rolling(window=12,center=False).mean())
        if tiposerie == 'FORE' or tiposerie=='TUTTO' : ax.plot(df_fore[header])
        self.canvasFN.draw()
        
        header = 'ST_'+tipocliente
        self.imgST.clear()
        ax = self.imgST.add_subplot(111,title=Decl['ST'])
        if tiposerie == 'CONS' or tiposerie=='TUTTO' :
            ax.plot(df_cons[header])
            ax.plot(df_cons[header].rolling(window=12,center=False).mean())
        if tiposerie == 'FORE' or tiposerie=='TUTTO' : ax.plot(df_fore[header])
        self.canvasST.draw()

    def ChangeGraph(self,*args):
        tipocliente = self.radiobase1.get()
        tiposerie   = self.radiobase2.get()
        self.RefreshGraph(tipocliente,tiposerie)
    
    def ForecastManager(self) :
        tkMessageBox.showinfo('FORECAST','Inizio Procedura')
        sequences = []
        for txt1 in Tipomisura_fore : 
            for txt2 in Tipocliente_file :
                sequences.append(txt1+'_'+txt2)
                
            #sequences = ('FL_IMP','CP_IMP','FL_PER','CP_PER','FL_AZR','CP_AZR','FL_RET','CP_RET')
    #TMS_value      = []
    #TMS_startgraph = []
    #TMS_endgraph   = []
        for TMS in sequences :
            TimeSerie = pd.Series(df_cons[TMS])
            df_fore[TMS], df_pred[TMS] = self.Forecast(TimeSerie, graphic=True)
        #TMS_value.append(listofforecast[0])
        #TMS_startgraph.append(listofforecast[1])
        #TMS_endgraph.append(listofforecast[3])
    
        for txt in Tipocliente_file :
            df_fore['FN_'+txt] = df_fore['FL_'+txt] - df_fore['CP_'+txt]
            df_pred['FN_'+txt] = df_pred['FL_'+txt] - df_pred['CP_'+txt]
    
        for txt in Tipomisura :
            df_fore[txt+'_TOTRET'] = df_fore[txt+'_AZR'] + df_fore[txt+'_RET']
            df_pred[txt+'_TOTRET'] = df_pred[txt+'_AZR'] + df_pred[txt+'_RET']
            df_fore[txt+'_BDT']    = df_fore[txt+'_IMP'] + df_fore[txt+'_PER'] + df_fore[txt+'_AZR'] + df_fore[txt+'_RET']
            df_pred[txt+'_BDT']    = df_pred[txt+'_IMP'] + df_pred[txt+'_PER'] + df_pred[txt+'_AZR'] + df_pred[txt+'_RET']

        df_fore.to_csv('NCFAfore.csv',sep=';')
        tkMessageBox.showinfo('FORECAST','Fine Procedura')
#        for TC in Tipocliente :
#            for TM in Tipomisura :
#                titolo = TM+'_'+TC
#                TMS = self.df_sstor[titolo]
#                TMS_rolmean = TMS.rolling(window=12,center=False).mean()
#                plt.plot(TMS,label='CONSUNTIVO')
#                plt.plot(TMS_rolmean,label='Media mobile')
#                plt.title(Decl[TM]+'  '+Decl[TC])
#                plt.legend()
#                self.Graphics[TC]['CONS'][TM] = plt.figure()
#                plt.clf

    def Forecast(self, TMS_orig = None, graphic = False):
        # Funzione che data una serie ne calcola il forecast
        # l'argomento è la serie storica originale, 
        def test_stationarity(tms = None) :
            # Dickey - Fuller test
            # H0 : la serie non è stazionaria
            # ritorna il p-value
            dftest = adfuller(tms, autolag='AIC')
        #Wlog(' ')
        #Wlog("Test Dickey-Fuller per verifica stazionarieta'")
        #Wlog('Serie temporale     : '+tms.name)
        #Wlog("H0 - ipotesi nulla  : La serie non e' stazionaria")
        #Wlog('p_value riscontrato : {:05.4f}'.format(dftest[1]))
            #if dftest[1]>alpha :
            #    Wlog("L'ipotesi nulla non si rifiuta - La serie NON e' stazionaria")
            #else:
            #    Wlog("L'ipotesi nulla viene rifiutata - La serie e' stazionaria")
            #Wlog(' ')
            return dftest[1]
        
        def test_whitenoise(tms = None) :   
            # Test Ljung-box sui residui per verifica White noise
            # (H0 : i residui sono WN)
            testWN = acorr_ljungbox(tms,lags=12)
            p_value_min = min(testWN[1])
        #Wlog(' ')
        #Wlog("Test Ljung-Box per verifica WHITE NOISE")
        #Wlog('Serie temporale     : '+tms.name)
        #Wlog("H0 - ipotesi nulla  : La serie e' white noise")
        #Wlog('p_value riscontrato : {:05.4f}'.format(p_value_min))
        #if p_value_min>alpha :
        #    Wlog("L'ipotesi nulla non si rifiuta - La serie e' White Noise")
        #else:
        #    Wlog("L'ipotesi nulla viene rifiutata - La serie NON e' White Noise")
        #Wlog(' ')
            return p_value_min
    
    #Wlog(' ')
    #Wlog('Iniziata procedura di FORECAST per la serie temporale '+TMS_orig.name)
    #Wlog(' ')
    #Wlog('Numero di osservazioni presenti      : '+str(len(TMS_orig)))
    
        #Elimina eventuali spazi vuoti
        TMS_orig = TMS_orig.dropna()
    #Wlog('Numero di osservazioni post-cleaning : '+str(len(TMS_orig)))
    
    #Calcola ed espone media mobile e deviazione standard su 12 periodi
    #TMS_rolmean = TMS_orig.rolling(window=12,center=False).mean()
    #TMS_rolstd  = TMS_orig.rolling(window=12,center=False).std()
        
    #plt.plot(TMS_orig, label='Original Data')
    #plt.plot(TMS_rolmean, label='Rolling Mean')
    #plt.plot(TMS_rolstd, label='Rolling Standard Dev.')
    #plt.title('Original series and Rolling Mean')
    #plt.legend()
    #start_graph = plt.figure()
    #if graphic : plt.show()
    #print start_graph
        # differenziazione per la stagionalita'
        TMS_diff_seas = TMS_orig - TMS_orig.shift(12) #effetto stagionalita' eliminato
        TMS_effetto_trend = TMS_diff_seas / 12        #effetto trend mensile
        #trend mensile medio sull'ultimo anno
        effetto_trend = np.mean(TMS_effetto_trend[-12:].values)
    
    #Wlog('Ricalcolata serie temporale depurata degli effetti stagionali')
    #Wlog("Ricalcolata serie temporale depurata dell'effetto trend")
    #Wlog('Effetto trend mensile (media degli ultimi 12 mesi)'+str(effetto_trend))
    
        decomposition = seasonal_decompose(TMS_orig,two_sided=False)
        TMS_trend     = decomposition.trend.dropna()
        TMS_seas      = decomposition.seasonal.dropna()
        TMS_resid     = decomposition.resid.dropna()
        TMS_resid.name = TMS_orig.name + "_residual"
    
    #plt.subplot(411)
    #plt.plot(TMS_orig, label='Original')
    #plt.legend(loc='best')
    #plt.subplot(412)
    #plt.plot(TMS_trend, label='Trend')
    #plt.legend(loc='best')
    #plt.subplot(413)
    #plt.plot(TMS_seas,label='Seasonality')
    #plt.legend(loc='best')
    #plt.subplot(414)
    #plt.plot(TMS_resid, label='Residuals')
    #plt.legend(loc='best')
    #plt.tight_layout()
    #decompose_graph = plt.figure()
    #if graphic : plt.show()
    
        # Ricerca il numero di differenziazioni sino a giungere alla stazionarietà
        max_diff  = 5
        TMS_for_test = TMS_resid
        rank_diff = 0
    
    #Wlog('Test per la verifica della stazionarietà sulla serie destagionalizzata e detrendizzata')
    
        while (test_stationarity(TMS_for_test) > alpha) and (rank_diff < max_diff) :
            rank_diff += 1
            TMS_for_test = TMS_for_test - TMS_for_test.shift()
            TMS_for_test = TMS_for_test.dropna()
            TMS_for_test.name = TMS_resid.name + '('+str(rank_diff)+')'
    
    #Wlog('Numero di differenziazioni da attuare per rendere la serie stazionaria : '+str(rank_diff))
    #Wlog('Se uguale a zero   : valore ammissibile ma piuttosto raro')
    #Wlog('Se uguale a uno    : valore standard e diffuso')
    #Wlog('Se uguale a due    : possono esserci trend anomali (almeno quadratici)')
    #Wlog('Se maggiore di due : valori anomali - serie temporale imprevedibile, anche se converge rischio di inferenza distorta')
    
    # se la serie è già stazionaria prova a vedere se si tratta di un white noise
        if rank_diff == 0 :
            if test_whitenoise(TMS_resid) > alpha : 
                whitenoise = True
            else :
                whitenoise = False
        else :
            whitenoise = False
        
        if whitenoise :
        #Wlog('Serie temporale uguale al white noise - nessuna ulteriore informazione catturabile')
            TMS_predict = pd.Series(0,index=df_cons.index)
            TMS_forecast = pd.Series(0,index=df_fore.index)
        else :
        # rappresentazione di ACF e PACF
        #lag_acf = acf(TMS_for_test, nlags=15)
        #lag_pacf = pacf(TMS_for_test, nlags=15, method='ols')
        #Plot ACF: 
        #plt.subplot(121) 
        #plt.plot(lag_acf)
        #plt.axhline(y=0,linestyle='--',color='gray')
        #plt.axhline(y=-1.96/np.sqrt(len(TMS_for_test)),linestyle='--',color='gray')
        #plt.axhline(y=1.96/np.sqrt(len(TMS_for_test)),linestyle='--',color='gray')
        #plt.title('Autocorrelation Function')
        #Plot PACF:
        #plt.subplot(122)
        #plt.plot(lag_pacf)
        #plt.axhline(y=0,linestyle='--',color='gray')
        #plt.axhline(y=-1.96/np.sqrt(len(TMS_for_test)),linestyle='--',color='gray')
        #plt.axhline(y=1.96/np.sqrt(len(TMS_for_test)),linestyle='--',color='gray')
        #plt.title('Partial Autocorrelation Function')
        #plt.tight_layout()
        #acfpacf_graph = plt.figure()
        #if graphic : plt.show()
    
            autoregressive      = TMS_resid
            autoregressive_diff = TMS_for_test
    
            criteria_ic = arma_order_select_ic(autoregressive_diff)
            rank_AR = criteria_ic.bic_min_order[0]
            rank_MA = criteria_ic.bic_min_order[1]
        #Wlog('Modello autoregressivo stimato : ARIMA('+str(rank_AR)+','+str(rank_diff)+','+str(rank_MA)+')')
    
            results_AR = ARIMA(autoregressive,order=(rank_AR,rank_diff,rank_MA)).fit(trend='c',disp=-1)
    
            TMS_predict = results_AR.fittedvalues
 
        #print results_AR.summary()
        #plt.plot(autoregressive_diff)
        #plt.plot(TMS_predict, color='red')
        #plt.title('RSS: %.4f'% sum((TMS_predict.dropna()-autoregressive_diff)**2))
        #plt.show()
    
            forecast = results_AR.forecast(steps=mesi_forecast)
        #print forecast
            TMS_forecast    = pd.Series(forecast[0],index=index_forecast)
            forecast_stderr = pd.Series(forecast[1],index=index_forecast)
   
        # FORECAST

        new_TMS_predict = TMS_predict + TMS_trend + TMS_seas

        new_TMS_trend = pd.Series(effetto_trend,index=index_forecast)
        new_TMS_trend[0] = new_TMS_trend[0] + TMS_trend[-1]
        new_TMS_trend = new_TMS_trend.cumsum()
    
        mask_seas = TMS_seas[-12:]
        new_TMS_seas = pd.Series(0,index=index_forecast)
        index_base = 0
        ranges = range(0,len(index_forecast))
        for index_base in ranges :
            index_mask = index_base % 12
            new_TMS_seas[index_base] = mask_seas[index_mask]
    
        # Serie di forecast finale
    
        new_TMS_forecast = TMS_forecast + new_TMS_trend + new_TMS_seas
    
    #plt.plot(TMS_orig,        color='blue', label='Valori osservati')
    #plt.plot(new_TMS_predict, color='red',  label='Valori predetti')
    #plt.plot(new_TMS_forecast,color='green',label='Valori forecast')
    #plt.legend(loc='best')
    #end_graph = plt.figure()
    #if graphic : plt.show()
    
    
    #new_TMS_forecast.name = 'Forecast for '+TMS_orig.name
    
    #print pd.concat([TMS_orig,TMS_trend,TMS_seas,TMS_resid],axis=1)
       #df_fore = pd.DataFrame.from_items([('value',TMS_forecast),('std error',forecast_stderr)])
        return new_TMS_forecast, new_TMS_predict #start_graph, decompose_graph, end_graph        

root = Tk()
App = Application(root)
root.title('New Customers Forecast Application - rel. 1.0 - (C) Gianni Zattoni')
root.mainloop()


#print 'Results of Dickey-Fuller Test:'
#dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
#for key,value in dftest[4].items():
#dfoutput['Critical Value (%s)'%key] = value
#print dfoutput
#    for text, mode in MODES:
#        b = Radiobutton(master, text=text,
#                        variable=v, value=mode)
#        b.pack(anchor=W)
#Radiobutton(master, text="One", variable=v, value=1).pack(anchor=W)
#Radiobutton(master, text="Two", variable=v, value=2).pack(anchor=W)


In [ ]:
# Funzione che data una serie ne calcola il forecast
# l'argomento è la serie storica originale, 
def Forecast(TMS_orig = None, graphic = False):
    
    def test_stationarity(tms = None) :
        # Dickey - Fuller test
        # H0 : la serie non è stazionaria
        # ritorna il p-value
        dftest = adfuller(tms, autolag='AIC')
        Wlog(' ')
        Wlog("Test Dickey-Fuller per verifica stazionarieta'")
        Wlog('Serie temporale     : '+tms.name)
        Wlog("H0 - ipotesi nulla  : La serie non e' stazionaria")
        Wlog('p_value riscontrato : {:05.4f}'.format(dftest[1]))
        if dftest[1]>alpha :
            Wlog("L'ipotesi nulla non si rifiuta - La serie NON e' stazionaria")
        else:
            Wlog("L'ipotesi nulla viene rifiutata - La serie e' stazionaria")
        Wlog(' ')
        return dftest[1]
        
        #print 'Results of Dickey-Fuller Test:'
        #dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
        #for key,value in dftest[4].items():
        #dfoutput['Critical Value (%s)'%key] = value
        #print dfoutput

    def test_whitenoise(tms = None) :   
        # Test Ljung-box sui residui per verifica White noise
        # (H0 : i residui sono WN)
        testWN = acorr_ljungbox(tms,lags=12)
        p_value_min = min(testWN[1])
        Wlog(' ')
        Wlog("Test Ljung-Box per verifica WHITE NOISE")
        Wlog('Serie temporale     : '+tms.name)
        Wlog("H0 - ipotesi nulla  : La serie e' white noise")
        Wlog('p_value riscontrato : {:05.4f}'.format(p_value_min))
        if p_value_min>alpha :
            Wlog("L'ipotesi nulla non si rifiuta - La serie e' White Noise")
        else:
            Wlog("L'ipotesi nulla viene rifiutata - La serie NON e' White Noise")
        Wlog(' ')
        return p_value_min
    
    Wlog(' ')
    Wlog('Iniziata procedura di FORECAST per la serie temporale '+TMS_orig.name)
    Wlog(' ')
    Wlog('Numero di osservazioni presenti      : '+str(len(TMS_orig)))
    
    #Elimina eventuali spazi vuoti
    TMS_orig = TMS_orig.dropna()
    Wlog('Numero di osservazioni post-cleaning : '+str(len(TMS_orig)))
    
    #La serie viene tagliata ai 4 anni più recenti e ...
    # + 12 mesi per la differenziazione stagionale e ...
    # + 1 mese per la differenziazione de-trend
    #if len(TMS_orig)>61 :
    #    TMS_orig = TMS_orig[-61:]
    #elif len(TMS_orig)>49 :
    #    TMS_orig = TMS_orig[-49:]
    #elif len(TMS_orig)>37 :
    #    TMS_orig = TMS_orig[-37:]
    #else :
    #    raise 0
    #Wlog('Numero di osservazioni che verranno prese in considerazione (più recenti) : '+str(len(TMS_orig)))
    
    #Calcola ed espone media mobile e deviazione standard su 12 periodi
    TMS_rolmean = TMS_orig.rolling(window=12,center=False).mean()
    #TMS_rolstd  = TMS_orig.rolling(window=12,center=False).std()
        
    plt.plot(TMS_orig, label='Original Data')
    plt.plot(TMS_rolmean, label='Rolling Mean')
    #plt.plot(TMS_rolstd, label='Rolling Standard Dev.')
    plt.title('Original series and Rolling Mean')
    plt.legend()
    start_graph = plt.figure()
    if graphic : plt.show()
    print start_graph
    # differenziazione per la stagionalita'
    TMS_diff_seas = TMS_orig - TMS_orig.shift(12) #effetto stagionalita' eliminato
    TMS_effetto_trend = TMS_diff_seas / 12        #effetto trend mensile
    #trend mensile medio sull'ultimo anno
    effetto_trend = np.mean(TMS_effetto_trend[-12:].values)
    
    Wlog('Ricalcolata serie temporale depurata degli effetti stagionali')
    Wlog("Ricalcolata serie temporale depurata dell'effetto trend")
    Wlog('Effetto trend mensile (media degli ultimi 12 mesi)'+str(effetto_trend))
    
    decomposition = seasonal_decompose(TMS_orig,two_sided=False)
    TMS_trend     = decomposition.trend.dropna()
    TMS_seas      = decomposition.seasonal.dropna()
    TMS_resid     = decomposition.resid.dropna()
    TMS_resid.name = TMS_orig.name + "_residual"
    
    plt.subplot(411)
    plt.plot(TMS_orig, label='Original')
    plt.legend(loc='best')
    plt.subplot(412)
    plt.plot(TMS_trend, label='Trend')
    plt.legend(loc='best')
    plt.subplot(413)
    plt.plot(TMS_seas,label='Seasonality')
    plt.legend(loc='best')
    plt.subplot(414)
    plt.plot(TMS_resid, label='Residuals')
    plt.legend(loc='best')
    plt.tight_layout()
    decompose_graph = plt.figure()
    if graphic : plt.show()
    
    # Ricerca il numero di differenziazioni sino a giungere alla stazionarietà
    max_diff  = 5
    TMS_for_test = TMS_resid
    rank_diff = 0
    
    Wlog('Test per la verifica della stazionarietà sulla serie destagionalizzata e detrendizzata')
    
    while (test_stationarity(TMS_for_test) > alpha) and (rank_diff < max_diff) :
        rank_diff += 1
        TMS_for_test = TMS_for_test - TMS_for_test.shift()
        TMS_for_test = TMS_for_test.dropna()
        TMS_for_test.name = TMS_resid.name + '('+str(rank_diff)+')'
    
    Wlog('Numero di differenziazioni da attuare per rendere la serie stazionaria : '+str(rank_diff))
    Wlog('Se uguale a zero   : valore ammissibile ma piuttosto raro')
    Wlog('Se uguale a uno    : valore standard e diffuso')
    Wlog('Se uguale a due    : possono esserci trend anomali (almeno quadratici)')
    Wlog('Se maggiore di due : valori anomali - serie temporale imprevedibile, anche se converge rischio di inferenza distorta')
    
    # se la serie è già stazionaria prova a vedere se si tratta di un white noise
    if rank_diff == 0 :
        if test_whitenoise(TMS_resid) > alpha : 
            whitenoise = True
        else :
            whitenoise = False
    else :
        whitenoise = False
        
    if whitenoise :
        Wlog('Serie temporale uguale al white noise - nessuna ulteriore informazione catturabile')
        TMS_predict = pd.Series(0,index=df_sstor.index)
        TMS_forecast = pd.Series(0,index=index_forecast)
    else :
        # rappresentazione di ACF e PACF
        lag_acf = acf(TMS_for_test, nlags=15)
        lag_pacf = pacf(TMS_for_test, nlags=15, method='ols')
        #Plot ACF: 
        plt.subplot(121) 
        plt.plot(lag_acf)
        plt.axhline(y=0,linestyle='--',color='gray')
        plt.axhline(y=-1.96/np.sqrt(len(TMS_for_test)),linestyle='--',color='gray')
        plt.axhline(y=1.96/np.sqrt(len(TMS_for_test)),linestyle='--',color='gray')
        plt.title('Autocorrelation Function')
        #Plot PACF:
        plt.subplot(122)
        plt.plot(lag_pacf)
        plt.axhline(y=0,linestyle='--',color='gray')
        plt.axhline(y=-1.96/np.sqrt(len(TMS_for_test)),linestyle='--',color='gray')
        plt.axhline(y=1.96/np.sqrt(len(TMS_for_test)),linestyle='--',color='gray')
        plt.title('Partial Autocorrelation Function')
        plt.tight_layout()
        acfpacf_graph = plt.figure()
        if graphic : plt.show()
    
        autoregressive      = TMS_resid
        autoregressive_diff = TMS_for_test
    
        criteria_ic = arma_order_select_ic(autoregressive_diff)
        rank_AR = criteria_ic.bic_min_order[0]
        rank_MA = criteria_ic.bic_min_order[1]
        Wlog('Modello autoregressivo stimato : ARIMA('+str(rank_AR)+','+str(rank_diff)+','+str(rank_MA)+')')
    
        results_AR = ARIMA(autoregressive,order=(rank_AR,rank_diff,rank_MA)).fit(trend='c',disp=-1)
    
        TMS_predict = results_AR.fittedvalues
 
        #print results_AR.summary()
        #plt.plot(autoregressive_diff)
        #plt.plot(TMS_predict, color='red')
        #plt.title('RSS: %.4f'% sum((TMS_predict.dropna()-autoregressive_diff)**2))
        #plt.show()
    
        forecast = results_AR.forecast(steps=mesi_forecast)
        #print forecast
        TMS_forecast    = pd.Series(forecast[0],index=index_forecast)
        forecast_stderr = pd.Series(forecast[1],index=index_forecast)
   
    # FORECAST

    new_TMS_predict = TMS_predict + TMS_trend + TMS_seas

    new_TMS_trend = pd.Series(effetto_trend,index=index_forecast)
    new_TMS_trend[0] = new_TMS_trend[0] + TMS_trend[-1]
    new_TMS_trend = new_TMS_trend.cumsum()
    
    mask_seas = TMS_seas[-12:]
    new_TMS_seas = pd.Series(0,index=index_forecast)
    index_base = 0
    ranges = range(0,len(index_forecast))
    for index_base in ranges :
        index_mask = index_base % 12
        new_TMS_seas[index_base] = mask_seas[index_mask]
    
    # Serie di forecast finale
    
    new_TMS_forecast = TMS_forecast + new_TMS_trend + new_TMS_seas
    
    plt.plot(TMS_orig,        color='blue', label='Valori osservati')
    plt.plot(new_TMS_predict, color='red',  label='Valori predetti')
    plt.plot(new_TMS_forecast,color='green',label='Valori forecast')
    plt.legend(loc='best')
    end_graph = plt.figure()
    if graphic : plt.show()
    
    
    new_TMS_forecast.name = 'Forecast for '+TMS_orig.name
    
    #print pd.concat([TMS_orig,TMS_trend,TMS_seas,TMS_resid],axis=1)
       #df_fore = pd.DataFrame.from_items([('value',TMS_forecast),('std error',forecast_stderr)])
    return new_TMS_forecast, start_graph, decompose_graph, end_graph